In [41]:
import pandas as pd
import numpy as np
import math
import graphviz as gz


In [42]:
def read_data(csv_file):
    largest_column_count =0
    with open(csv_file, 'r') as temp_f:
        lines = temp_f.readlines()
        for l in lines:
            column_count = len(l.split(','))
            #找到列数最多的行
            largest_column_count = column_count if largest_column_count < column_count else largest_column_count
    temp_f.close()
    # colunm_names为最大列数展开
    column_names = [str(i) for i in range(0, largest_column_count)]
    data = pd.read_csv(csv_file, header=None, delimiter=',', names=column_names)
    data.fillna(0)
    return data

In [43]:
chipResourceData = read_data('./../../data/attachment1.csv')
variableDependData = read_data('../../data/attachment2.csv')
blockRelationshipData = read_data('../../data/attachment3.csv')

In [44]:
# variableDependData.head(10)
chipResourceData.head(10)

,0,1,2,3,4
0,BLOCK,TCAM,HASH,ALU,QUALIFY
1,0,0,0,2,0
2,1,0,0,0,0
3,2,0,0,0,0
4,3,0,0,0,0
5,4,0,0,10,3
6,5,0,0,6,0
7,6,0,0,0,0
8,7,0,0,3,4
9,8,0,0,4,4


In [45]:
def convert_int(data):
    data.fillna(-1, inplace=True)
    data[list(data.columns)] = data[list(data.columns)].astype(int)
    return data

blockRelationshipData = convert_int(blockRelationshipData)
variableDependData = variableDependData.fillna(-1)




In [46]:

blockRealtionshipMatrix = np.ndarray(shape=(607,607), dtype=int)
blockRelationshipData = blockRelationshipData.values.tolist()
edges = []
for i in blockRelationshipData:
    for j in range(1,12):
        if i[j] != -1:
            blockRealtionshipMatrix[i][j] = 1
            edges.append([str(i[0]), str(i[j])])

dot = gz.Digraph()
for index in range(607):
    dot.node(str(index), str(index))
dot.edges(edges)
dot.format = 'svg'
dot.render()

'Digraph.gv.svg'

In [47]:
variableDependData.head(10)
variableDependData[variableDependData['0'].isin([0,1])]

,0,1,2,3,4,5,6,7,8,9,...,38,39,40,41,42,43,44,45,46,47
0,0,W,X0,X1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,0,R,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,1,W,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,1,R,-1,-1,-1,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [50]:
dataDependency = np.ndarray(shape=(607,607),dtype=int) # 无数据依赖 0， 读后写数据依赖1，写后写数据依赖与写后读数据依赖均为2  

In [57]:
def get_dependency_data(rows, columns):
    data =  variableDependData[variableDependData['0'].isin([rows, columns])]
    data.drop(['1'],axis=1,inplace=True)
    data =  data.values.tolist()
    res = 0 #数据依赖表示 0为没有数据依赖 读后写依赖为1 写后读依赖为2
    for i in range(len(data)):
        while -1 in data[i]:
            data[i].remove(-1)
    if (set.intersection(set(data[0]),set(data[2])) != set()) or (set.intersection(set(data[0]),set(data[3])) != set()): # 写后读或写后写
        res = 2
    if (set.intersection(set(data[1]),set(data[2])) != set()) or (set.intersection(set(data[1]),set(data[3])) != set()): # 读后写
        res = 1
    return res

for rows in range(len(dataDependency)):
    for columns in range(len(dataDependency[rows])):
        if rows == columns:
            dataDependency[rows][columns] = 0
        else:
            dataDependency[rows][columns] = get_dependency_data(rows, columns)
print(dataDependency)

c:\Users\YSHuaWei\.conda\envs\d2l_learn\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]


In [58]:
np.save("dataDependency", dataDependency)

In [61]:
dataDependency[10]

array([0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,

In [ ]:
# 考虑贪心算法？